In [1]:
import numpy as np 
import pandas as pd 

In [2]:
exclude_genes = pd.read_csv("./list_of_excluded_genes.csv")
raw_df = pd.read_csv("../../Final_expression_051820/Sample_final_df_RawExp.csv")
full_mapping = pd.read_table('./ATCC_CFT073_K12MG_K12DH_BW_gene_map.txt')
full_mapping.columns = ['ATCC', 'ATCC_names', 'BW', 'BW_names', 'DH', 'DH_names', 'MG', 'MG_names', 'drop']
full_mapping = full_mapping[['ATCC_names','BW_names', 'DH_names', 'MG_names']]
full_mapping = full_mapping.drop_duplicates(keep='first') 

In [4]:
#1130 genes to exclude
ATCC_exclude = exclude_genes['ATCC_names'].drop_duplicates().to_numpy()
DH_exclude = exclude_genes['DH_names'].drop_duplicates().to_numpy()
BW_exclude = exclude_genes['BW_names'].drop_duplicates().to_numpy()
MG_exclude = exclude_genes['MG_names'].drop_duplicates().to_numpy()

In [68]:
# filter individual strains
ATCC_raw = raw_df[raw_df['strain']=='ATCC'] #3181
BW_raw = raw_df[raw_df['strain']=='BW25113'] #3040
DH_raw = raw_df[raw_df['strain']=='K12DH'] #3058
MG_raw = raw_df[raw_df['strain']=='K12MG'] #2969

#### exclude list of genes for every strain

In [69]:
ATCC_new = ATCC_raw[~ATCC_raw['gene_id'].isin(ATCC_exclude)].reset_index(drop=True) #2232
BW_new = BW_raw[~BW_raw['gene_id'].isin(BW_exclude)].reset_index(drop=True) #3040
DH_new = DH_raw[~DH_raw['gene_id'].isin(DH_exclude)].reset_index(drop=True) #3055
MG_new = MG_raw[~MG_raw['gene_id'].isin(MG_exclude)].reset_index(drop=True) #2969

In [70]:
ATCC_new.rename(columns={'Locus_tag':'gene_id', 'gene_id':'Locus_tag'}, inplace=True)

In [71]:
dfs = [ATCC_new, BW_new, DH_new, MG_new]

for i in dfs: 
    i['exp_name'] = i['strain'] + '_' + i['replicates']
    i.drop(['strain','gbk_start', 'gbk_end', 'gbk_midpoint','gbk_strand',
            'block', 'start', 'end', 'taxa', 'replicates', 'exp', 'gene_id'], axis=1, inplace=True)

In [73]:
mapping = full_mapping[~full_mapping['ATCC_names'].isin(ATCC_exclude)]
mapping = mapping[~mapping['BW_names'].isin(BW_exclude)]

#### fixing DH

In [151]:
DH_info = pd.read_table("../Ecoli_K12_DH10B_NC_010473_gene_info.txt")

In [158]:
DH_new = pd.merge(DH_new, DH_info[['gbk_locus_tag','gbk_old_locus_tag']],
                 left_on='Locus_tag', right_on='gbk_old_locus_tag', how='left')

In [161]:
DH_new.drop(['Locus_tag','gbk_old_locus_tag'], axis=1, inplace=True)
DH_new.rename(columns={'gbk_locus_tag':'Locus_tag'}, inplace=True)

### spreading dataframes

In [164]:
ATCC_exp = ATCC_new.pivot(index='Locus_tag',columns='exp_name',values='raw_exp').reset_index()
ATCC_rev_comp = ATCC_new.pivot(index='Locus_tag',columns='exp_name',values='rev_comp').reset_index()
ATCC_inversion = ATCC_new.pivot(index='Locus_tag',columns='exp_name',values='inversion').reset_index()

In [165]:
BW_exp = BW_new.pivot(index='Locus_tag',columns='exp_name',values='raw_exp').reset_index()
BW_rev_comp = BW_new.pivot(index='Locus_tag',columns='exp_name',values='rev_comp').reset_index()
BW_inversion = BW_new.pivot(index='Locus_tag',columns='exp_name',values='inversion').reset_index()

In [166]:
DH_exp = DH_new.pivot(index='Locus_tag',columns='exp_name',values='raw_exp').reset_index()
DH_rev_comp = DH_new.pivot(index='Locus_tag',columns='exp_name',values='rev_comp').reset_index()
DH_inversion = DH_new.pivot(index='Locus_tag',columns='exp_name',values='inversion').reset_index()

In [167]:
MG_exp = MG_new.pivot(index='Locus_tag',columns='exp_name',values='raw_exp').reset_index()
MG_rev_comp = MG_new.pivot(index='Locus_tag',columns='exp_name',values='rev_comp').reset_index()
MG_inversion = MG_new.pivot(index='Locus_tag',columns='exp_name',values='inversion').reset_index()

### expression values

In [168]:
exp = pd.merge(mapping, ATCC_exp, left_on='ATCC_names', right_on='Locus_tag') 
exp.drop(['Locus_tag'], axis=1, inplace=True) #2039

In [169]:
exp = pd.merge(exp, BW_exp, left_on='MG_names', right_on='Locus_tag')
exp.drop(['Locus_tag'], axis=1, inplace=True) #2040

In [170]:
exp = pd.merge(exp, DH_exp, left_on='DH_names', right_on='Locus_tag')
exp.drop(['Locus_tag'], axis=1, inplace=True) #1985

In [172]:
exp = pd.merge(exp, MG_exp, left_on='MG_names', right_on='Locus_tag')
exp.drop(['Locus_tag'], axis=1, inplace=True) #1936

In [173]:
len(exp)

1936

### rev_comp values

In [174]:
rev_comp = pd.merge(mapping, ATCC_rev_comp, left_on='ATCC_names', right_on='Locus_tag') 
rev_comp.drop(['Locus_tag'], axis=1, inplace=True) #2039

In [175]:
rev_comp = pd.merge(rev_comp, BW_rev_comp, left_on='MG_names', right_on='Locus_tag')
rev_comp.drop(['Locus_tag'], axis=1, inplace=True) #2040

In [176]:
rev_comp = pd.merge(rev_comp, DH_rev_comp, left_on='DH_names', right_on='Locus_tag')
rev_comp.drop(['Locus_tag'], axis=1, inplace=True) #2024

In [177]:
rev_comp = pd.merge(rev_comp, MG_rev_comp, left_on='MG_names', right_on='Locus_tag')
rev_comp.drop(['Locus_tag'], axis=1, inplace=True) #1975

In [178]:
len(rev_comp)

1936

### inversion values

In [179]:
inversion = pd.merge(mapping, ATCC_inversion, left_on='ATCC_names', right_on='Locus_tag') 
inversion.drop(['Locus_tag'], axis=1, inplace=True) #2039

In [180]:
inversion = pd.merge(inversion, BW_inversion, left_on='MG_names', right_on='Locus_tag')
inversion.drop(['Locus_tag'], axis=1, inplace=True) #2040

In [181]:
inversion = pd.merge(inversion, DH_inversion, left_on='DH_names', right_on='Locus_tag')
inversion.drop(['Locus_tag'], axis=1, inplace=True) #2024

In [182]:
inversion = pd.merge(inversion, MG_inversion, left_on='MG_names', right_on='Locus_tag')
inversion.drop(['Locus_tag'], axis=1, inplace=True) #1975

In [183]:
len(inversion)

1936

### export new dfs

In [187]:
exp.to_csv('./raw_exp.csv', index=False)

In [188]:
rev_comp.to_csv('./raw_rev_comp.csv', index=False)

In [189]:
inversion.to_csv('./raw_inversion.csv', index=False)

In [193]:
len(exp) == len(rev_comp) == len(inversion)

True